<h1> Data Spaces' Tesina <h1>

<p>Imports section<p>

In [1]:
import numpy as np
from sklearn import neighbors, model_selection, metrics, preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

<p> Read .csv file <p>

In [2]:
# filename = "dataR2.csv"
filename = "breast-cancer-wisconsin.data"
file = open(filename, "r")

# data = np.loadtxt(file, delimiter=",", dtype=None, encoding=None, usecols=(0,1,2,3,4,5,6,7,8,9), skiprows=1)

data = np.loadtxt(file, delimiter=",", dtype=None, encoding=None, usecols=(1,2,3,4,5,6,7,8,9, 10), skiprows=0)
# record with 0 are indeed missing valeues => re-download data and see how to treat missing values better 

# records with the classification as "1" are Healthy Controls, "2" means Patients.

<p>Preprocessing</p>

In [3]:
# Let's divide data into source and target (respectively X and Y)
X = data[:, :-1]
Y = data[:, len(data[0])-1]

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, random_state=np.random.randint(0,100), test_size=0.3)

# array of possible K to apply KNN neighbors
ks = [3,5,7,9]


# normalize data because algorithms work better with normalized data
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# print(X)

<p> I try to apply PCA on the dataset, since there are 10 features (which are a lot) and then re-apply KNN <p>

In [4]:
# pca = PCA(n_components=2)
# pca.fit(X)
# Xp = pca.transform(X)
# print(Xp)

# X_train, X_test, Y_train, Y_test = model_selection.train_test_split(Xp, Y, random_state=np.random.randint(0,100), test_size=0.3)

# # array of possible K to apply KNN neighbors
# ks = [3,5,7,9]


# # normalize data because algorithms work better with normalized data
# scaler = preprocessing.StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

<p>Apply K-NN</p>

In [5]:
for k in ks:
    n_neighbors = k
    
    # Create an instance of neighbors classifier (clf) and fit the data
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    
    # train the classifier on the training set
    clf.fit(X_train, Y_train)
    
    print("Accuracy score on the test set with K =",n_neighbors,"is %.3f" %(clf.score(X_test, Y_test)))


    

Accuracy score on the test set with K = 3 is 0.971
Accuracy score on the test set with K = 5 is 0.971
Accuracy score on the test set with K = 7 is 0.971
Accuracy score on the test set with K = 9 is 0.971


<p>Apply KNN this time with a validation set </p>

In [6]:
# Let's create the validation set
X_train_t, X_valid, Y_train_t, Y_valid = model_selection.train_test_split(X_train, Y_train, random_state=np.random.randint(0,100), test_size=0.30)

# array of possible K to apply KNN neighbors
ks = [3,5,7,9]

# normalize data because algorithms work better with normalized data
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

# array to write accuracy values for each K 
acc_arr = []

for k in ks:
    n_neighbors = k
    
    # Create an instance of neighbors classifier (clf) and fit the data
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    
    # train the classifier on the training set
    clf.fit(X_train_t, Y_train_t)
    
    acc_arr.append(clf.score(X_valid, Y_valid))
    
# I choose the best K based on the results on the validation set and apply KNN with that
k_best_index = acc_arr.index(max(acc_arr))
k_best = ks[k_best_index]

clf2 = neighbors.KNeighborsClassifier(k_best)
clf2.fit(X_train_t, Y_train_t)

print("The best K, based on the validation set results, is",k_best,"and the accuracy on the test set is %.3f"%(clf2.score(X_test, Y_test)))

The best K, based on the validation set results, is 5 and the accuracy on the test set is 0.971


<p> I'll try Logistic Regression on the dataset <p>

In [7]:
logReg = LogisticRegression(solver="lbfgs") # instance of the model

logReg.fit(X_train, Y_train)

# res = logReg.predict(X_test)

print("Accuracy: ", logReg.score(X_test, Y_test))

Accuracy:  0.9714285714285714


<p><u> Naive Bayes classifier </u></p>

In [8]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
print("Accuracy: ", gnb.score(X_test, Y_test))

Accuracy:  0.9619047619047619


<p><u> Random forest</u> </p>

In [9]:
rndFor = RandomForestClassifier(n_estimators = 200, criterion="entropy")

rndFor.fit(X_train, Y_train)

y_pred = rndFor.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(Y_test, y_pred))

Accuracy:  0.9714285714285714


<p><u> SVM</u> </p>

In [10]:
svc = SVC(gamma='auto', kernel='rbf')
svc.fit(X_train, Y_train)
print("Accuracy: ", svc.score(X_test, Y_test))

Accuracy:  0.9666666666666667


<p> So this is the accuracy with the default parameters, but with SVm approach is necessary to perform some parameters tuning in order to achieve a better result. To do so I relied on the GridSearch <p>
<p><u> SVM with GridSearch </u></p>

In [11]:
parameters = [{'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1], 'kernel': ['rbf']}]
svcGS = GridSearchCV(svc, parameters, n_jobs=-1, cv=3)
sm = SMOTE(random_state=12)
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train)
svcGS.fit(X_train_res, Y_train_res)
print("Accuracy: ", svcGS.score(X_test, Y_test))

Accuracy:  0.9714285714285714
